## 1.机器翻译

机器翻译(MT)是指将一段文本从一种语言自动翻译为另一种语言，其主要特征：输出单词序列，且输出序列的长度与源序列的长度不同。

其主要步骤：

(1)清洗数据将其转换为神经网络输入minbatch

(2)分词

(3)建立词典

(4)使用Encoder-Decoder，encoder将输入转换为隐藏状态，decoder将隐藏状态转换为输出

## 2.注意力机制

Seq2Seq的解码器在各个时间步依赖相同的北京变量来获取输入序列信息。当编码器为循环神经⽹络时，背景变量来⾃它最终时间步的隐藏状态。将源序列输入信息以循环单位状态编码，然后将其传递给解码器以生成目标序列。然而这种结构存在着问题，尤其是RNN机制实际中存在长程梯度消失的问题，对于较长的句子，我们很难寄希望于将输入的序列转化为定长的向量而保存所有的有效信息，所以随着所需翻译句子的长度的增加，这种结构的效果会显著下降。

与此同时，解码的目标词语可能只与原输入的部分词语有关，而并不是与所有的输入有关，seq2seq模型中，解码器只能隐式地从编码器的最终状态中选择相应的信息。然而，注意力机制可以将这种选择过程显式地建模。

### 注意力机制

一种通用的带权池化方法。输入由两部分构成：询问（query）和键值对（key-value pairs）。对于一个query来说，attention layer 会与每一个key计算注意力分数并进行权重的归一化，输出的向量 o 则是value的加权求和，而每个key计算的权重与value一一对应。

不同的attetion layer的区别在于score函数的选择

### 2.1 卷积注意力

假设query和keys有相同的维度, 即  ∀i,q,k_i∈Rd . 通过计算query和key转置的乘积来计算attention score,通常还会除去sqrt(d)减少计算出来的score对维度𝑑的依赖性，如下
                          
                          α(q,k)=⟨q,k⟩/sqrt(d)
                          
### 2.2 多层感知机注意力

将score函数定义

                            α(k,q)=v的转置* tanh(W_k*k+W_q*q)
                            
然后将key 和 value 在特征的维度上合并，然后送至单隐层MLP，隐层为ℎ，输出的size为1，隐层激活函数为tanh，无偏置.

## 3.Transformer

(1)CNN易于并行化，但不适合捕捉长序列内的依赖关系

(2)RNN适合捕捉长序列的依赖关系，但是难以实现并行化处理序列

整合CNN和RNN的优势,结合注意力机制设计了Transformer模型。该模型利用attention机制实现了并行化捕捉序列依赖，并且同时处理序列的每个位置的tokens。

Transformer同样基于编码器-解码器架构，其区别主要在于以下三点：

(1)Transformer blocks：将seq2seq模型中的循环网络替换为了Transformer Blocks，该模块包含一个多头注意力层（Multi-head Attention Layers）以及两个position-wise feed-forward networks（FFN）。对于解码器来说，另一个多头注意力层被用于接受编码器的隐藏状态。

(2)Add and norm：多头注意力层和前馈网络的输出被送到两个“add and norm”层进行处理，该层包含残差结构以及层归一化。

(3)Position encoding：由于自注意力层并没有区分元素的顺序，所以一个位置编码层被用于向序列元素里添加位置信息。

### Transformer Block

(1)多头注意力层

多头注意力层包含 h 个并行的自注意力层，每一个这种层被称为一个head。对每个头来说，在进行注意力计算之前，我们会将query、key和value用三个现行层进行映射，这 h 个注意力头的输出将会被拼接之后输入最后一个线性层进行整合。

(2)基于位置的前馈网络（FFN）

它接受一个形状为（batch_size，seq_length, feature_size）的三维张量。Position-wise FFN由两个全连接层组成，他们作用在最后一维上。因为序列的每个位置的状态都会被单独地更新，所以我们称他为position-wise，这等效于一个1x1的卷积。